# Pipeline de Data Engineering et Machine Learning

Ce notebook démontre un pipeline complet de data engineering et machine learning, depuis la collecte de données jusqu'au déploiement d'une API pour servir les prédictions.

Le pipeline comprend les étapes suivantes :
1. Collecte de données (IMDb, Twitter, CSV)
2. Nettoyage des données
3. Feature Engineering
4. Entraînement d'un modèle avec scikit-learn
5. Stockage du modèle avec joblib
6. Déploiement d'une API avec FastAPI

Suivez ce notebook pour comprendre et exécuter chaque étape du processus.

## Installation des dépendances

Commençons par installer les bibliothèques nécessaires pour notre pipeline :

In [ ]:
# Installer les bibliothèques nécessaires
# Décommentez les lignes suivantes si vous avez besoin d'installer les packages
# !pip install numpy pandas scikit-learn matplotlib seaborn requests beautifulsoup4 joblib fastapi uvicorn tweepy nltk

# Importation des bibliothèques principales
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration des visualisations
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(style="whitegrid")
%matplotlib inline

# Définir le répertoire du projet directement à la racine
project_dir = os.path.abspath(os.path.join('c:/xampp/htdocs/projet a faire pour mdmaine'))
if project_dir not in sys.path:
    sys.path.append(project_dir)

print(f"Répertoire du projet : {project_dir}")
print(f"Python version : {sys.version}")
print(f"Pandas version : {pd.__version__}")
print(f"NumPy version : {np.__version__}")

# Créer les répertoires de données s'ils n'existent pas déjà
os.makedirs(os.path.join(project_dir, 'data', 'raw'), exist_ok=True)
os.makedirs(os.path.join(project_dir, 'data', 'processed'), exist_ok=True)
os.makedirs(os.path.join(project_dir, 'data', 'processed', 'features'), exist_ok=True)
os.makedirs(os.path.join(project_dir, 'models'), exist_ok=True)
os.makedirs(os.path.join(project_dir, 'src'), exist_ok=True)
os.makedirs(os.path.join(project_dir, 'api'), exist_ok=True)
os.makedirs(os.path.join(project_dir, 'notebooks'), exist_ok=True)

## 1. Collecte de données

Dans cette section, nous allons collecter des données à partir de différentes sources :
- IMDb (données de films)
- Twitter (données de tweets)
- Fichiers CSV locaux

Nous utiliserons les modules que nous avons développés dans le dossier `src` du projet.

In [ ]:
# Importer le module de collecte de données
from src.data_collection import collect_imdb_data, collect_twitter_data, load_csv_data

# Définir le chemin des données brutes
RAW_DATA_DIR = os.path.join(project_dir, 'data', 'raw')

### 1.1 Collecte de données IMDb

Récupérons des données sur quelques films populaires depuis IMDb :

In [ ]:
# Liste d'identifiants IMDb de films populaires
movie_ids = [
    'tt0111161',  # Les Évadés (The Shawshank Redemption)
    'tt0068646',  # Le Parrain (The Godfather)
    'tt0071562',  # Le Parrain, 2e partie (The Godfather: Part II)
    'tt0468569',  # The Dark Knight
    'tt0050083',  # 12 Hommes en colère (12 Angry Men)
    'tt0108052',  # La Liste de Schindler (Schindler's List)
    'tt0167260',  # Le Seigneur des anneaux : Le Retour du roi (LOTR: Return of the King)
    'tt0110912',  # Pulp Fiction
    'tt0060196',  # Le Bon, la Brute et le Truand (The Good, the Bad and the Ugly)
    'tt0137523'   # Fight Club
]

# Collecter les données
imdb_data = collect_imdb_data(movie_ids)

# Afficher les premières lignes du DataFrame
if not imdb_data.empty:
    display(imdb_data.head())
    print(f"Nombre de films collectés : {imdb_data.shape[0]}")
    print(f"Nombre de colonnes : {imdb_data.shape[1]}")
else:
    print("Aucune donnée n'a été collectée. Vérifiez votre clé API ou votre connexion internet.")
    
    # Créer un exemple de données de films pour la démonstration si la collecte échoue
    imdb_data = pd.DataFrame({
        'Title': ['The Shawshank Redemption', 'The Godfather', 'The Dark Knight', 'Pulp Fiction', 'Fight Club'],
        'Year': ['1994', '1972', '2008', '1994', '1999'],
        'Rated': ['R', 'R', 'PG-13', 'R', 'R'],
        'Released': ['14 Oct 1994', '24 Mar 1972', '18 Jul 2008', '14 Oct 1994', '15 Oct 1999'],
        'Runtime': ['142 min', '175 min', '152 min', '154 min', '139 min'],
        'Genre': ['Drama', 'Crime, Drama', 'Action, Crime, Drama', 'Crime, Drama', 'Drama'],
        'Director': ['Frank Darabont', 'Francis Ford Coppola', 'Christopher Nolan', 'Quentin Tarantino', 'David Fincher'],
        'Writer': ['Stephen King, Frank Darabont', 'Mario Puzo, Francis Ford Coppola', 'Jonathan Nolan, Christopher Nolan', 'Quentin Tarantino', 'Chuck Palahniuk, Jim Uhls'],
        'Actors': ['Tim Robbins, Morgan Freeman', 'Marlon Brando, Al Pacino', 'Christian Bale, Heath Ledger', 'John Travolta, Uma Thurman', 'Brad Pitt, Edward Norton'],
        'Plot': ['Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.', 'When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.', 'The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.', 'An insomniac office worker and a devil-may-care soapmaker form an underground fight club that evolves into something much, much more.'],
        'Language': ['English', 'English, Italian, Latin', 'English', 'English, Spanish, French', 'English'],
        'Country': ['USA', 'USA', 'USA, UK', 'USA', 'USA, Germany'],
        'Awards': ['Nominated for 7 Oscars. Another 21 wins & 32 nominations.', 'Won 3 Oscars. Another 24 wins & 28 nominations.', 'Won 2 Oscars. Another 153 wins & 159 nominations.', 'Won 1 Oscar. Another 70 wins & 75 nominations.', 'Nominated for 1 Oscar. Another 11 wins & 37 nominations.'],
        'Poster': ['https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_SX300.jpg', 'https://m.media-amazon.com/images/M/MV5BM2MyNjYxNmUtYTAwNi00MTYxLWJmNWYtYzZlODY3ZTk3OTFlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg', 'https://m.media-amazon.com/images/M/MV5BMTMxNTMwODM0NF5BMl5BanBnXkFtZTcwODAyMTk2Mw@@._V1_SX300.jpg', 'https://m.media-amazon.com/images/M/MV5BNGNhMDIzZTUtNTBlZi00MTRlLWFjM2ItYzViMjE3YzI5MjljXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg', 'https://m.media-amazon.com/images/M/MV5BMmEzNTkxYjQtZTc0MC00YTVjLTg5ZTEtZWMwOWVlYzY0NWIwXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg'],
        'Ratings': [{'Source': 'Internet Movie Database', 'Value': '9.3/10'}, {'Source': 'Internet Movie Database', 'Value': '9.2/10'}, {'Source': 'Internet Movie Database', 'Value': '9.0/10'}, {'Source': 'Internet Movie Database', 'Value': '8.9/10'}, {'Source': 'Internet Movie Database', 'Value': '8.8/10'}],
        'Metascore': ['80', '100', '84', '94', '66'],
        'imdbRating': ['9.3', '9.2', '9.0', '8.9', '8.8'],
        'imdbVotes': ['2,400,000', '1,700,000', '2,350,000', '1,900,000', '1,850,000'],
        'imdbID': ['tt0111161', 'tt0068646', 'tt0468569', 'tt0110912', 'tt0137523'],
        'Type': ['movie', 'movie', 'movie', 'movie', 'movie'],
        'DVD': ['21 Dec 1999', '11 Oct 2001', '09 Dec 2008', '19 May 1998', '14 Jun 2000'],
        'BoxOffice': ['$28,767,189', '$135,000,000', '$534,858,444', '$107,928,762', '$37,030,102'],
        'Production': ['Columbia Pictures, Castle Rock Entertainment', 'Paramount Pictures', 'Warner Bros., Legendary Entertainment', 'Miramax Films', '20th Century Fox, Regency Enterprises'],
        'Website': ['N/A', 'N/A', 'N/A', 'N/A', 'N/A']
    })
    
    # Sauvegarde du jeu de données d'exemple
    imdb_data.to_csv(os.path.join(RAW_DATA_DIR, 'imdb_data.csv'), index=False)
    display(imdb_data.head())
    print("Un jeu de données d'exemple a été créé pour la démonstration.")

### 1.2 Collecte de données Twitter

Collectons maintenant des tweets sur un sujet spécifique (par exemple "data science") :

In [ ]:
# Collecter des tweets sur le thème "data science"
query = "data science"

try:
    # Cette fonction nécessite des clés d'API Twitter configurées
    twitter_data = collect_twitter_data(query, count=100)
    
    # Afficher les premières lignes si des données ont été collectées
    if not twitter_data.empty:
        display(twitter_data.head())
        print(f"Nombre de tweets collectés : {twitter_data.shape[0]}")
    else:
        raise ValueError("Aucun tweet collecté")
        
except Exception as e:
    print(f"Erreur lors de la collecte des tweets : {str(e)}")
    print("Création d'un jeu de données d'exemple pour la démonstration...")
    
    # Créer un exemple de données de tweets pour la démonstration
    twitter_data = pd.DataFrame({
        'id': ['1373256789', '1373256790', '1373256791', '1373256792', '1373256793'],
        'created_at': pd.date_range(start='2023-01-01', periods=5),
        'text': [
            "I love working with #DataScience projects! The insights you can gain from data are amazing. #AI #ML",
            "Just finished my latest machine learning model with scikit-learn. 95% accuracy! #DataScience #Python",
            "Data Science is transforming every industry. Companies need to adapt or get left behind. #DataScience #DigitalTransformation",
            "Attending a great webinar on data visualization techniques. So many ways to tell stories with data! #DataScience #DataViz",
            "Struggling with this neural network architecture. Anyone have tips for image classification? #DeepLearning #DataScience"
        ],
        'user': ['data_enthusiast', 'ml_expert', 'tech_journalist', 'data_viz_pro', 'ai_student'],
        'retweets': [42, 78, 25, 18, 5],
        'favorites': [156, 234, 87, 56, 12],
        'hashtags': [
            ['DataScience', 'AI', 'ML'],
            ['DataScience', 'Python'],
            ['DataScience', 'DigitalTransformation'],
            ['DataScience', 'DataViz'],
            ['DeepLearning', 'DataScience']
        ]
    })
    
    # Sauvegarde du jeu de données d'exemple
    twitter_data.to_csv(os.path.join(RAW_DATA_DIR, 'twitter_data_science.csv'), index=False)
    display(twitter_data.head())
    print("Un jeu de données d'exemple a été créé pour la démonstration.")

### 1.3 Chargement de données CSV

Démontrons comment charger des données à partir d'un fichier CSV :

In [ ]:
# Créer un petit dataset d'exemple si besoin
sample_data_path = os.path.join(RAW_DATA_DIR, 'sample_data.csv')

if not os.path.exists(sample_data_path):
    # Créer un jeu de données d'exemple
    sample_data = pd.DataFrame({
        'user_id': range(1, 101),
        'age': np.random.randint(18, 65, 100),
        'gender': np.random.choice(['M', 'F', 'Other'], 100),
        'income': np.random.normal(50000, 15000, 100),
        'education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], 100),
        'movie_genre_preference': np.random.choice(['Action', 'Comedy', 'Drama', 'Sci-Fi', 'Horror'], 100),
        'rating_frequency': np.random.randint(1, 50, 100),
        'last_active': pd.date_range(start='2023-01-01', periods=100)
    })
    
    # Ajouter quelques valeurs manquantes
    sample_data.loc[np.random.choice(sample_data.index, 10), 'income'] = np.nan
    sample_data.loc[np.random.choice(sample_data.index, 5), 'age'] = np.nan
    
    # Sauvegarder les données
    sample_data.to_csv(sample_data_path, index=False)
    print(f"Jeu de données d'exemple créé et sauvegardé dans {sample_data_path}")
else:
    print(f"Le fichier {sample_data_path} existe déjà")

# Charger les données avec notre fonction
csv_data = load_csv_data(sample_data_path)

# Afficher les premières lignes
display(csv_data.head())
print(f"Nombre d'enregistrements : {csv_data.shape[0]}")
print(f"Nombre de colonnes : {csv_data.shape[1]}")

# Afficher des informations sur les données
print("\nInformations sur les données :")
print(csv_data.info())

# Statistiques descriptives
print("\nStatistiques descriptives :")
display(csv_data.describe())

## 2. Nettoyage des données

Dans cette section, nous allons nettoyer les données collectées en :
- Gérant les valeurs manquantes
- Supprimant les doublons
- Normalisant les formats
- Convertissant les types de données

Nous utiliserons les modules que nous avons développés dans le dossier `src` du projet.

In [ ]:
# Importer le module de nettoyage des données
from src.data_cleaning import clean_imdb_data, clean_twitter_data, clean_csv_data

# Définir les chemins des répertoires
PROCESSED_DATA_DIR = os.path.join(project_dir, 'data', 'processed')

### 2.1 Nettoyage des données IMDb

Nettoyons les données IMDb collectées précédemment :

In [ ]:
# Nettoyer les données IMDb
clean_imdb = clean_imdb_data('imdb_data.csv')

# Afficher les données nettoyées
if clean_imdb is not None:
    display(clean_imdb.head())
    
    # Visualiser les valeurs manquantes
    plt.figure(figsize=(12, 6))
    sns.heatmap(clean_imdb.isnull(), cbar=False, yticklabels=False, cmap='viridis')
    plt.title('Valeurs manquantes dans les données IMDb nettoyées')
    plt.tight_layout()
    plt.show()
    
    # Comparaison avant/après pour certaines colonnes
    if 'imdbRating' in clean_imdb.columns and 'imdbRating' in imdb_data.columns:
        fig, ax = plt.subplots(1, 2, figsize=(12, 5))
        
        # Avant nettoyage
        ax[0].hist(pd.to_numeric(imdb_data['imdbRating'], errors='coerce'), bins=10, alpha=0.7)
        ax[0].set_title('Notes IMDb avant nettoyage')
        ax[0].set_xlabel('Note')
        ax[0].set_ylabel('Fréquence')
        
        # Après nettoyage
        ax[1].hist(clean_imdb['imdbRating'], bins=10, alpha=0.7, color='green')
        ax[1].set_title('Notes IMDb après nettoyage')
        ax[1].set_xlabel('Note')
        
        plt.tight_layout()
        plt.show()
else:
    print("Échec du nettoyage des données IMDb.")

### 2.2 Nettoyage des données Twitter

Nettoyons maintenant les données Twitter :

In [ ]:
# Identifier le fichier Twitter à nettoyer
twitter_files = [f for f in os.listdir(RAW_DATA_DIR) if f.startswith('twitter_')]

if twitter_files:
    # Nettoyer le premier fichier trouvé
    twitter_file = twitter_files[0]
    clean_twitter = clean_twitter_data(twitter_file)
    
    if clean_twitter is not None:
        display(clean_twitter.head())
        
        # Visualiser les longueurs de tweets avant et après nettoyage
        if 'text' in twitter_data.columns and 'clean_text' in clean_twitter.columns:
            fig, ax = plt.subplots(1, 2, figsize=(14, 6))
            
            # Avant nettoyage
            tweet_lengths = twitter_data['text'].str.len()
            ax[0].hist(tweet_lengths, bins=20, alpha=0.7)
            ax[0].set_title('Longueur des tweets avant nettoyage')
            ax[0].set_xlabel('Nombre de caractères')
            ax[0].set_ylabel('Fréquence')
            
            # Après nettoyage
            clean_tweet_lengths = clean_twitter['clean_text'].str.len()
            ax[1].hist(clean_tweet_lengths, bins=20, alpha=0.7, color='green')
            ax[1].set_title('Longueur des tweets après nettoyage')
            ax[1].set_xlabel('Nombre de caractères')
            
            plt.tight_layout()
            plt.show()
            
            # Comparer un exemple de tweet avant et après nettoyage
            print("Exemple de tweet avant et après nettoyage :")
            sample_idx = 0
            original_tweet = twitter_data.iloc[sample_idx]['text']
            cleaned_tweet = clean_twitter.iloc[sample_idx]['clean_text']
            
            print(f"Original: {original_tweet}")
            print(f"Nettoyé : {cleaned_tweet}")
else:
    print("Aucun fichier Twitter trouvé pour le nettoyage.")
    print("Création d'un exemple de données Twitter nettoyées...")
    
    # Créer un exemple de données Twitter nettoyées
    clean_twitter = pd.DataFrame({
        'id': ['1373256789', '1373256790', '1373256791', '1373256792', '1373256793'],
        'created_at': pd.date_range(start='2023-01-01', periods=5),
        'text': [
            "I love working with #DataScience projects! The insights you can gain from data are amazing. #AI #ML",
            "Just finished my latest machine learning model with scikit-learn. 95% accuracy! #DataScience #Python",
            "Data Science is transforming every industry. Companies need to adapt or get left behind. #DataScience #DigitalTransformation",
            "Attending a great webinar on data visualization techniques. So many ways to tell stories with data! #DataScience #DataViz",
            "Struggling with this neural network architecture. Anyone have tips for image classification? #DeepLearning #DataScience"
        ],
        'clean_text': [
            "I love working with DataScience projects The insights you can gain from data are amazing",
            "Just finished my latest machine learning model with scikitlearn 95 accuracy",
            "Data Science is transforming every industry Companies need to adapt or get left behind",
            "Attending a great webinar on data visualization techniques So many ways to tell stories with data",
            "Struggling with this neural network architecture Anyone have tips for image classification"
        ],
        'user': ['data_enthusiast', 'ml_expert', 'tech_journalist', 'data_viz_pro', 'ai_student'],
        'retweets': [42, 78, 25, 18, 5],
        'favorites': [156, 234, 87, 56, 12],
        'tweet_length': [107, 92, 105, 112, 115]
    })
    
    # Sauvegarde dans le dossier processed
    clean_twitter.to_csv(os.path.join(PROCESSED_DATA_DIR, 'clean_twitter_data_science.csv'), index=False)
    display(clean_twitter.head())
    print("Un exemple de données Twitter nettoyées a été créé pour la démonstration.")

### 2.3 Nettoyage des données CSV

Nettoyons maintenant les données CSV avec des transformations spécifiques :

In [ ]:
# Vérifier si le fichier sample_data.csv existe
if os.path.exists(os.path.join(RAW_DATA_DIR, 'sample_data.csv')):
    # Définir les configurations de nettoyage
    date_columns = ['last_active']
    numeric_columns = ['age', 'income', 'rating_frequency']
    categorical_columns = ['gender', 'education', 'movie_genre_preference']
    
    # Nettoyer les données
    clean_sample_data = clean_csv_data(
        'sample_data.csv', 
        date_columns=date_columns,
        numeric_columns=numeric_columns,
        categorical_columns=categorical_columns
    )
    
    if clean_sample_data is not None:
        display(clean_sample_data.head())
        
        # Vérifier les types de données après nettoyage
        print("Types de données après nettoyage :")
        print(clean_sample_data.dtypes)
        
        # Visualiser l'effet du nettoyage sur les valeurs manquantes
        fig, ax = plt.subplots(1, 2, figsize=(14, 6))
        
        # Avant nettoyage
        sns.heatmap(csv_data.isnull(), cbar=False, yticklabels=False, cmap='viridis', ax=ax[0])
        ax[0].set_title('Valeurs manquantes avant nettoyage')
        
        # Après nettoyage
        sns.heatmap(clean_sample_data.isnull(), cbar=False, yticklabels=False, cmap='viridis', ax=ax[1])
        ax[1].set_title('Valeurs manquantes après nettoyage')
        
        plt.tight_layout()
        plt.show()
        
        # Visualiser la distribution d'une variable numérique
        if 'income' in clean_sample_data.columns:
            plt.figure(figsize=(10, 6))
            sns.histplot(clean_sample_data['income'], kde=True)
            plt.title('Distribution des revenus après nettoyage')
            plt.xlabel('Revenu')
            plt.ylabel('Fréquence')
            plt.tight_layout()
            plt.show()
else:
    print("Fichier sample_data.csv non trouvé.")

## 3. Feature Engineering

Dans cette section, nous allons créer de nouvelles caractéristiques à partir des données nettoyées pour améliorer les performances des modèles. Les transformations incluent :
- Encodage des variables catégorielles
- Extraction de caractéristiques à partir du texte
- Normalisation et standardisation des données
- Création de caractéristiques dérivées

Nous utiliserons les modules que nous avons développés dans le dossier `src` du projet.

In [ ]:
# Importer le module de feature engineering
from src.feature_engineering import engineer_imdb_features, engineer_twitter_features, engineer_custom_features

# Définir le chemin du répertoire des caractéristiques
FEATURES_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, 'features')

### 3.1 Feature Engineering pour les données IMDb

Créons de nouvelles caractéristiques pour les données IMDb :

In [ ]:
# Vérifier si le fichier de données nettoyées existe
if os.path.exists(os.path.join(PROCESSED_DATA_DIR, 'clean_imdb_data.csv')):
    # Créer de nouvelles caractéristiques
    featured_imdb = engineer_imdb_features()
    
    if featured_imdb is not None:
        # Afficher les données avec les nouvelles caractéristiques
        print("Caractéristiques des données IMDb après feature engineering :")
        print(f"Nombre de lignes : {featured_imdb.shape[0]}")
        print(f"Nombre de colonnes : {featured_imdb.shape[1]}")
        print("\nNouvelles colonnes :")
        
        # Identifier les nouvelles colonnes ajoutées
        clean_columns = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, 'clean_imdb_data.csv')).columns.tolist()
        new_columns = [col for col in featured_imdb.columns if col not in clean_columns]
        
        print(new_columns)
        
        # Afficher les premières lignes
        display(featured_imdb[new_columns].head())
        
        # Visualiser quelques nouvelles caractéristiques
        if 'movie_age' in featured_imdb.columns and 'imdbRating' in featured_imdb.columns:
            plt.figure(figsize=(10, 6))
            plt.scatter(featured_imdb['movie_age'], featured_imdb['imdbRating'], alpha=0.7)
            plt.title('Relation entre l\'âge du film et sa note IMDb')
            plt.xlabel('Âge du film (années)')
            plt.ylabel('Note IMDb')
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()
        
        # Visualiser la distribution d'une nouvelle caractéristique
        if 'plot_sentiment' in featured_imdb.columns:
            plt.figure(figsize=(10, 6))
            sns.histplot(featured_imdb['plot_sentiment'], kde=True)
            plt.title('Distribution du sentiment des résumés de films')
            plt.xlabel('Score de sentiment (-1 = négatif, 1 = positif)')
            plt.ylabel('Fréquence')
            plt.tight_layout()
            plt.show()
            
            # Relation entre le sentiment du résumé et la note
            if 'imdbRating' in featured_imdb.columns:
                plt.figure(figsize=(10, 6))
                plt.scatter(featured_imdb['plot_sentiment'], featured_imdb['imdbRating'], alpha=0.7)
                plt.title('Relation entre le sentiment du résumé et la note IMDb')
                plt.xlabel('Score de sentiment du résumé')
                plt.ylabel('Note IMDb')
                plt.grid(True, linestyle='--', alpha=0.7)
                plt.tight_layout()
                plt.show()
else:
    print("Le fichier clean_imdb_data.csv n'existe pas. Impossible de procéder au feature engineering.")

### 3.2 Feature Engineering pour les données Twitter

Créons de nouvelles caractéristiques pour les données Twitter :

In [ ]:
# Rechercher les fichiers Twitter nettoyés
clean_twitter_files = [f for f in os.listdir(PROCESSED_DATA_DIR) if f.startswith('clean_twitter_')]

if clean_twitter_files:
    # Prendre le premier fichier trouvé
    clean_twitter_file = clean_twitter_files[0]
    
    # Générer les caractéristiques
    featured_twitter = engineer_twitter_features(clean_twitter_file)
    
    if featured_twitter is not None:
        # Afficher les données avec les nouvelles caractéristiques
        print("Caractéristiques des données Twitter après feature engineering :")
        print(f"Nombre de lignes : {featured_twitter.shape[0]}")
        print(f"Nombre de colonnes : {featured_twitter.shape[1]}")
        print("\nNouvelles colonnes :")
        
        # Identifier les nouvelles colonnes ajoutées
        clean_columns = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, clean_twitter_file)).columns.tolist()
        new_columns = [col for col in featured_twitter.columns if col not in clean_columns]
        
        print(new_columns)
        
        # Afficher les premières lignes
        display(featured_twitter[new_columns].head())
        
        # Visualiser les scores de sentiment
        if 'sentiment_score' in featured_twitter.columns:
            plt.figure(figsize=(10, 6))
            sns.histplot(featured_twitter['sentiment_score'], kde=True, bins=20)
            plt.title('Distribution des scores de sentiment des tweets')
            plt.xlabel('Score de sentiment (-1 = négatif, 1 = positif)')
            plt.ylabel('Fréquence')
            plt.axvline(x=0, color='red', linestyle='--')
            plt.tight_layout()
            plt.show()
            
            # Visualiser la relation entre le sentiment et l'engagement
            if 'engagement' in featured_twitter.columns:
                plt.figure(figsize=(10, 6))
                plt.scatter(featured_twitter['sentiment_score'], featured_twitter['engagement'], alpha=0.7)
                plt.title('Relation entre le sentiment et l\'engagement')
                plt.xlabel('Score de sentiment')
                plt.ylabel('Engagement (retweets + favoris)')
                plt.grid(True, linestyle='--', alpha=0.7)
                plt.tight_layout()
                plt.show()
        
        # Visualiser la répartition des tweets par heure de la journée
        if 'hour_of_day' in featured_twitter.columns:
            plt.figure(figsize=(12, 6))
            sns.countplot(data=featured_twitter, x='hour_of_day')
            plt.title('Répartition des tweets par heure de la journée')
            plt.xlabel('Heure de la journée')
            plt.ylabel('Nombre de tweets')
            plt.xticks(range(0, 24))
            plt.tight_layout()
            plt.show()
else:
    print("Aucun fichier Twitter nettoyé trouvé. Création de données d'exemple...")
    
    # Créer un exemple de données Twitter avec caractéristiques
    featured_twitter = pd.DataFrame({
        'id': ['1373256789', '1373256790', '1373256791', '1373256792', '1373256793'],
        'created_at': pd.date_range(start='2023-01-01', periods=5),
        'text': [
            "I love working with #DataScience projects! The insights you can gain from data are amazing. #AI #ML",
            "Just finished my latest machine learning model with scikit-learn. 95% accuracy! #DataScience #Python",
            "Data Science is transforming every industry. Companies need to adapt or get left behind. #DataScience #DigitalTransformation",
            "Attending a great webinar on data visualization techniques. So many ways to tell stories with data! #DataScience #DataViz",
            "Struggling with this neural network architecture. Anyone have tips for image classification? #DeepLearning #DataScience"
        ],
        'clean_text': [
            "I love working with DataScience projects The insights you can gain from data are amazing",
            "Just finished my latest machine learning model with scikitlearn 95 accuracy",
            "Data Science is transforming every industry Companies need to adapt or get left behind",
            "Attending a great webinar on data visualization techniques So many ways to tell stories with data",
            "Struggling with this neural network architecture Anyone have tips for image classification"
        ],
        'user': ['data_enthusiast', 'ml_expert', 'tech_journalist', 'data_viz_pro', 'ai_student'],
        'retweets': [42, 78, 25, 18, 5],
        'favorites': [156, 234, 87, 56, 12],
        'hour_of_day': [9, 14, 11, 16, 22],
        'day_of_week': [0, 2, 4, 1, 6],
        'is_weekend': [0, 0, 0, 0, 1],
        'sentiment_score': [0.78, 0.65, 0.42, 0.56, -0.23],
        'sentiment_positive': [1, 1, 1, 1, 0],
        'sentiment_negative': [0, 0, 0, 0, 1],
        'sentiment_neutral': [0, 0, 0, 0, 0],
        'word_count': [15, 11, 13, 16, 12],
        'has_question': [0, 0, 0, 0, 1],
        'has_exclamation': [1, 1, 0, 1, 0],
        'capital_letter_ratio': [0.08, 0.05, 0.12, 0.06, 0.04],
        'engagement': [198, 312, 112, 74, 17],
        'retweet_to_favorite_ratio': [0.27, 0.33, 0.29, 0.32, 0.42]
    })
    
    # Sauvegarde dans le dossier des caractéristiques
    os.makedirs(FEATURES_DATA_DIR, exist_ok=True)
    featured_twitter.to_csv(os.path.join(FEATURES_DATA_DIR, 'featured_twitter_data_science.csv'), index=False)
    
    display(featured_twitter.head())
    print("Un exemple de données Twitter avec caractéristiques a été créé pour la démonstration.")
    
    # Visualiser quelques caractéristiques
    plt.figure(figsize=(10, 6))
    sns.histplot(featured_twitter['sentiment_score'], kde=True, bins=10)
    plt.title('Distribution des scores de sentiment des tweets (données d\'exemple)')
    plt.xlabel('Score de sentiment (-1 = négatif, 1 = positif)')
    plt.ylabel('Fréquence')
    plt.axvline(x=0, color='red', linestyle='--')
    plt.tight_layout()
    plt.show()

### 3.3 Feature Engineering pour les données CSV

Créons maintenant des caractéristiques personnalisées pour nos données CSV :

In [ ]:
# Vérifier si le fichier de données nettoyées existe
clean_sample_csv = 'clean_sample_data.csv'
if os.path.exists(os.path.join(PROCESSED_DATA_DIR, clean_sample_csv)):
    # Définir la configuration de feature engineering
    config = {
        'scale_columns': ['age', 'income', 'rating_frequency'],
        'onehot_columns': ['gender', 'education', 'movie_genre_preference'],
        'bin_columns': [
            {'column': 'age', 'bins': 4},
            {'column': 'income', 'bins': 5}
        ],
        'interactions': [
            ('age', 'rating_frequency'),
            ('income', 'rating_frequency')
        ]
    }
    
    # Générer les caractéristiques
    featured_csv = engineer_custom_features(clean_sample_csv, config)
    
    if featured_csv is not None:
        # Afficher les données avec les nouvelles caractéristiques
        print("Caractéristiques des données CSV après feature engineering :")
        print(f"Nombre de lignes : {featured_csv.shape[0]}")
        print(f"Nombre de colonnes : {featured_csv.shape[1]}")
        print("\nNouvelles colonnes :")
        
        # Identifier les nouvelles colonnes ajoutées
        clean_columns = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, clean_sample_csv)).columns.tolist()
        new_columns = [col for col in featured_csv.columns if col not in clean_columns]
        
        print(new_columns)
        
        # Afficher les premières lignes
        display(featured_csv[new_columns].head())
        
        # Visualiser les variables standardisées
        if 'age_scaled' in featured_csv.columns and 'income_scaled' in featured_csv.columns:
            plt.figure(figsize=(10, 6))
            plt.scatter(featured_csv['age_scaled'], featured_csv['income_scaled'], alpha=0.7)
            plt.title('Variables standardisées: Âge vs Revenu')
            plt.xlabel('Âge (standardisé)')
            plt.ylabel('Revenu (standardisé)')
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()
        
        # Visualiser les variables discrétisées (binned)
        if 'age_binned' in featured_csv.columns:
            plt.figure(figsize=(10, 6))
            sns.countplot(data=featured_csv, x='age_binned')
            plt.title('Répartition des tranches d\'âge')
            plt.xlabel('Tranche d\'âge')
            plt.ylabel('Nombre d\'utilisateurs')
            plt.tight_layout()
            plt.show()
        
        # Visualiser le one-hot encoding d'une variable catégorielle
        onehot_cols = [col for col in new_columns if col.startswith('gender_')]
        if onehot_cols:
            onehot_sum = featured_csv[onehot_cols].sum()
            plt.figure(figsize=(10, 6))
            onehot_sum.plot(kind='bar')
            plt.title('Répartition des genres après one-hot encoding')
            plt.xlabel('Genre')
            plt.ylabel('Nombre d\'utilisateurs')
            plt.tight_layout()
            plt.show()
else:
    print(f"Le fichier {clean_sample_csv} n'existe pas. Impossible de procéder au feature engineering.")

## 4. Entraînement d'un modèle avec scikit-learn

Dans cette section, nous allons entraîner un modèle de machine learning en utilisant scikit-learn. Nous allons :
- Préparer les données pour l'entraînement (division train/test)
- Sélectionner et entraîner un modèle
- Évaluer les performances du modèle
- Analyser les caractéristiques importantes

Nous utiliserons les modules que nous avons développés dans le dossier `src` du projet.

In [ ]:
# Importer le module d'entraînement de modèle
from src.model_training import load_training_data, train_regression_model, train_classification_model, save_model

# Importer scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score

# Définir le chemin des modèles
MODELS_DIR = os.path.join(project_dir, 'models')

### 4.1 Entraînement d'un modèle de régression pour prédire les notes IMDb

Entraînons un modèle de régression pour prédire les notes IMDb à partir des caractéristiques générées :

In [ ]:
# Vérifier si le fichier de données avec caractéristiques existe
imdb_feature_file = 'featured_imdb_data.csv'
imdb_feature_path = os.path.join(FEATURES_DATA_DIR, imdb_feature_file)

if os.path.exists(imdb_feature_path):
    # Charger les données
    X_train, X_test, y_train, y_test, features = load_training_data(
        imdb_feature_file,
        target_column='imdbRating',
        features=None,  # Utiliser toutes les colonnes numériques
        test_size=0.2,
        random_state=42
    )
    
    if X_train is not None:
        print(f"Données d'entraînement: {X_train.shape[0]} exemples, {X_train.shape[1]} caractéristiques")
        print(f"Données de test: {X_test.shape[0]} exemples, {X_test.shape[1]} caractéristiques")
        
        # Entraîner différents modèles de régression
        models = ['linear', 'random_forest', 'gradient_boosting']
        performances = {}
        best_model = None
        best_score = -float('inf')
        
        for model_type in models:
            print(f"\nEntraînement du modèle {model_type}...")
            model, performance = train_regression_model(
                X_train, y_train, X_test, y_test, features,
                model_type=model_type
            )
            
            performances[model_type] = performance
            
            # Conserver le meilleur modèle selon le R²
            if performance['r2'] > best_score:
                best_model = model
                best_score = performance['r2']
                best_model_type = model_type
        
        # Comparer les performances des modèles
        print("\nComparaison des performances des modèles :")
        metrics = ['mse', 'rmse', 'r2']
        models_df = pd.DataFrame({
            model_type: [performances[model_type][metric] for metric in metrics]
            for model_type in models
        }, index=metrics)
        
        display(models_df)
        
        # Visualiser les performances
        plt.figure(figsize=(10, 6))
        models_df.loc['r2'].plot(kind='bar')
        plt.title('Comparaison des scores R² des modèles')
        plt.ylabel('Score R²')
        plt.ylim(0, 1)
        plt.tight_layout()
        plt.show()
        
        # Sauvegarder le meilleur modèle
        if best_model is not None:
            model_path = save_model(best_model, 'imdb_rating_predictor', {
                'performance': performances[best_model_type],
                'features': features,
                'target': 'imdbRating',
                'model_type': best_model_type
            })
            
            print(f"\nMeilleur modèle ({best_model_type}) sauvegardé dans {model_path}")
else:
    print(f"Le fichier {imdb_feature_path} n'existe pas. Impossible d'entraîner le modèle.")
    
    # Créer un jeu de données fictif pour la démonstration
    print("Création d'un jeu de données d'exemple pour la démonstration...")
    
    # Générer des données synthétiques
    np.random.seed(42)
    n_samples = 100
    
    X = np.random.rand(n_samples, 5)
    y = 5 + 2 * X[:, 0] + 3 * X[:, 1] - 1.5 * X[:, 2] + 0.5 * X[:, 3] + np.random.normal(0, 0.5, n_samples)
    
    feature_names = ['movie_age', 'actor_count', 'runtime', 'budget', 'popularity']
    X_df = pd.DataFrame(X, columns=feature_names)
    y_series = pd.Series(y, name='imdbRating')
    
    # Division en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X_df, y_series, test_size=0.2, random_state=42)
    
    print(f"Données synthétiques créées: {X_train.shape[0]} exemples d'entraînement, {X_test.shape[0]} exemples de test")
    
    # Entraîner un modèle de forêt aléatoire
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Évaluer le modèle
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Performance du modèle synthétique:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R²: {r2:.4f}")
    
    # Visualiser les prédictions vs valeurs réelles
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel('Valeurs réelles')
    plt.ylabel('Prédictions')
    plt.title('Prédictions vs Valeurs réelles (données synthétiques)')
    plt.tight_layout()
    plt.show()
    
    # Importance des caractéristiques
    feature_importances = model.feature_importances_
    features_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
    features_df = features_df.sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    plt.bar(features_df['feature'], features_df['importance'])
    plt.title('Importance des caractéristiques (données synthétiques)')
    plt.xlabel('Caractéristique')
    plt.ylabel('Importance')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Sauvegarder le modèle
    os.makedirs(MODELS_DIR, exist_ok=True)
    model_path = os.path.join(MODELS_DIR, 'demo_imdb_rating_predictor.joblib')
    
    import joblib
    joblib.dump(model, model_path)
    print(f"Modèle de démonstration sauvegardé dans {model_path}")

### 4.2 Entraînement d'un modèle de classification pour le sentiment des tweets

Entraînons un modèle de classification pour prédire le sentiment des tweets :

In [ ]:
# Rechercher les fichiers Twitter avec caractéristiques
twitter_feature_files = [f for f in os.listdir(FEATURES_DATA_DIR) if f.startswith('featured_') and 'twitter' in f.lower()]

if twitter_feature_files:
    # Prendre le premier fichier trouvé
    twitter_feature_file = twitter_feature_files[0]
    
    # Vérifier si la cible existe
    twitter_df = pd.read_csv(os.path.join(FEATURES_DATA_DIR, twitter_feature_file))
    
    if 'sentiment_positive' in twitter_df.columns:
        # Charger les données
        X_train, X_test, y_train, y_test, features = load_training_data(
            twitter_feature_file,
            target_column='sentiment_positive',
            features=None,  # Utiliser toutes les colonnes numériques
            test_size=0.2,
            random_state=42
        )
        
        if X_train is not None:
            print(f"Données d'entraînement: {X_train.shape[0]} exemples, {X_train.shape[1]} caractéristiques")
            print(f"Données de test: {X_test.shape[0]} exemples, {X_test.shape[1]} caractéristiques")
            
            # Entraîner différents modèles de classification
            models = ['logistic', 'random_forest', 'gradient_boosting']
            performances = {}
            best_model = None
            best_score = -float('inf')
            
            for model_type in models:
                print(f"\nEntraînement du modèle {model_type}...")
                model, performance = train_classification_model(
                    X_train, y_train, X_test, y_test, features,
                    model_type=model_type
                )
                
                performances[model_type] = performance
                
                # Conserver le meilleur modèle selon le F1-score
                if performance['f1'] > best_score:
                    best_model = model
                    best_score = performance['f1']
                    best_model_type = model_type
            
            # Comparer les performances des modèles
            print("\nComparaison des performances des modèles :")
            metrics = ['accuracy', 'precision', 'recall', 'f1']
            models_df = pd.DataFrame({
                model_type: [performances[model_type][metric] for metric in metrics]
                for model_type in models
            }, index=metrics)
            
            display(models_df)
            
            # Visualiser les performances
            plt.figure(figsize=(12, 6))
            models_df.plot(kind='bar')
            plt.title('Comparaison des performances des modèles')
            plt.ylabel('Score')
            plt.ylim(0, 1)
            plt.tight_layout()
            plt.show()
            
            # Sauvegarder le meilleur modèle
            if best_model is not None:
                model_path = save_model(best_model, 'twitter_sentiment_classifier', {
                    'performance': performances[best_model_type],
                    'features': features,
                    'target': 'sentiment_positive',
                    'model_type': best_model_type
                })
                
                print(f"\nMeilleur modèle ({best_model_type}) sauvegardé dans {model_path}")
    else:
        print("La colonne 'sentiment_positive' n'existe pas dans les données Twitter.")
else:
    print("Aucun fichier Twitter avec caractéristiques trouvé.")
    
    # Créer un jeu de données fictif pour la démonstration
    print("Création d'un jeu de données d'exemple pour la démonstration...")
    
    # Générer des données synthétiques
    np.random.seed(42)
    n_samples = 100
    
    X = np.random.rand(n_samples, 4)
    # Générer une cible binaire avec une relation logique
    y = (0.8 * X[:, 0] - 0.5 * X[:, 1] + 0.3 * X[:, 2] - 0.1 * X[:, 3] > 0.5).astype(int)
    
    feature_names = ['sentiment_score', 'word_count', 'capital_letter_ratio', 'has_exclamation']
    X_df = pd.DataFrame(X, columns=feature_names)
    y_series = pd.Series(y, name='sentiment_positive')
    
    # Division en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X_df, y_series, test_size=0.2, random_state=42)
    
    print(f"Données synthétiques créées: {X_train.shape[0]} exemples d'entraînement, {X_test.shape[0]} exemples de test")
    
    # Entraîner un modèle de forêt aléatoire
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Évaluer le modèle
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"Performance du modèle synthétique:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1 Score: {f1:.4f}")
    
    # Visualiser la matrice de confusion
    from sklearn.metrics import confusion_matrix
    
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Prédictions')
    plt.ylabel('Valeurs réelles')
    plt.title('Matrice de confusion (données synthétiques)')
    plt.tight_layout()
    plt.show()
    
    # Importance des caractéristiques
    feature_importances = model.feature_importances_
    features_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
    features_df = features_df.sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    plt.bar(features_df['feature'], features_df['importance'])
    plt.title('Importance des caractéristiques (données synthétiques)')
    plt.xlabel('Caractéristique')
    plt.ylabel('Importance')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Sauvegarder le modèle
    os.makedirs(MODELS_DIR, exist_ok=True)
    model_path = os.path.join(MODELS_DIR, 'demo_twitter_sentiment_classifier.joblib')
    
    import joblib
    joblib.dump(model, model_path)
    print(f"Modèle de démonstration sauvegardé dans {model_path}")

## 5. Stockage du modèle avec joblib

Dans cette section, nous allons explorer comment stocker et charger des modèles entraînés à l'aide de la bibliothèque joblib.

In [ ]:
# Importer joblib
import joblib
from datetime import datetime

# Créer un modèle simple pour la démonstration si nécessaire
if not os.path.exists(os.path.join(MODELS_DIR, 'demo_imdb_rating_predictor.joblib')):
    # Créer un modèle de forêt aléatoire simple
    from sklearn.ensemble import RandomForestRegressor
    model = RandomForestRegressor(n_estimators=50, random_state=42)
    
    # Entraîner sur des données synthétiques
    np.random.seed(42)
    X = np.random.rand(100, 5)
    y = 2 * X[:, 0] + 3 * X[:, 1] - X[:, 2] + np.random.normal(0, 0.1, 100)
    
    model.fit(X, y)
    
    # Sauvegarder dans le dossier des modèles
    os.makedirs(MODELS_DIR, exist_ok=True)
    joblib.dump(model, os.path.join(MODELS_DIR, 'demo_imdb_rating_predictor.joblib'))
    print("Modèle de démonstration créé.")

# Lister les modèles disponibles
model_files = [f for f in os.listdir(MODELS_DIR) if f.endswith('.joblib')]
print(f"Modèles disponibles: {model_files}")

# Sélectionner un modèle à charger (le premier disponible)
if model_files:
    selected_model = model_files[0]
    model_path = os.path.join(MODELS_DIR, selected_model)
    
    print(f"\nChargement du modèle: {selected_model}")
    
    # Mesurer le temps de chargement
    start_time = datetime.now()
    loaded_model = joblib.load(model_path)
    end_time = datetime.now()
    
    loading_time = (end_time - start_time).total_seconds()
    print(f"Modèle chargé en {loading_time:.4f} secondes.")
    
    # Afficher des informations sur le modèle
    print(f"Type du modèle: {type(loaded_model).__name__}")
    
    if hasattr(loaded_model, 'feature_importances_'):
        print("\nImportance des caractéristiques:")
        importances = loaded_model.feature_importances_
        
        # Si le modèle est une pipeline, extraire le composant final
        if hasattr(loaded_model, 'named_steps'):
            if 'model' in loaded_model.named_steps:
                importances = loaded_model.named_steps['model'].feature_importances_
        
        # Afficher les importances
        for i, importance in enumerate(importances):
            print(f"  Caractéristique {i}: {importance:.4f}")
    
    # Tester le modèle avec des données aléatoires
    print("\nTest du modèle avec des données aléatoires:")
    
    np.random.seed(42)
    X_test = np.random.rand(5, loaded_model.n_features_in_)
    
    # Faire des prédictions
    predictions = loaded_model.predict(X_test)
    
    # Afficher les prédictions
    for i, pred in enumerate(predictions):
        print(f"  Échantillon {i}: {pred:.4f}")
    
    # Taille du fichier modèle
    model_size = os.path.getsize(model_path) / (1024 * 1024)  # en Mo
    print(f"\nTaille du fichier modèle: {model_size:.2f} Mo")
    
    # Afficher les métadonnées si disponibles
    metadata_path = model_path.replace('.joblib', '_metadata.json')
    if os.path.exists(metadata_path):
        import json
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        
        print("\nMétadonnées du modèle:")
        for key, value in metadata.items():
            if key != 'performance':  # Afficher les performances séparément
                print(f"  {key}: {value}")
        
        if 'performance' in metadata:
            print("\nPerformance du modèle:")
            for metric, score in metadata['performance'].items():
                if metric != 'best_params':  # Afficher les paramètres séparément
                    print(f"  {metric}: {score:.4f}")
else:
    print("Aucun modèle disponible dans le dossier des modèles.")

## 6. Déploiement d'une API avec FastAPI

Dans cette section, nous allons voir comment déployer un modèle entraîné via une API FastAPI. Nous allons :
- Créer une application FastAPI
- Définir des routes pour les prédictions
- Tester l'API localement

Notez que le code complet de l'API est déjà disponible dans le dossier `api/` du projet.

In [ ]:
# Examiner le code de l'API
api_main_path = os.path.join(project_dir, 'api', 'main.py')

if os.path.exists(api_main_path):
    print("Aperçu du code de l'API FastAPI :")
    
    with open(api_main_path, 'r') as f:
        code = f.readlines()
    
    # Afficher les imports et l'initialisation de l'application
    for i, line in enumerate(code):
        if i < 30:  # Afficher les 30 premières lignes
            print(line.rstrip())
        elif i == 30:
            print("...")
else:
    print("Le fichier de l'API n'existe pas à l'emplacement attendu.")

### 6.1 Exemple minimal d'API FastAPI

Créons un exemple minimal d'API FastAPI pour servir un modèle :

In [ ]:
# Cet exemple montre comment créer une API FastAPI simple pour servir un modèle
# Dans un environnement réel, vous exécuteriez ce code dans un fichier séparé

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
import joblib
import numpy as np
import os
from typing import List, Dict, Any

# Définir le modèle de données d'entrée
class MovieInput(BaseModel):
    """Modèle pour les entrées de prédiction de film."""
    title: str = Field(..., example="The Shawshank Redemption")
    year: int = Field(..., example=1994)
    director: str = Field(..., example="Frank Darabont")
    actors: str = Field(..., example="Tim Robbins, Morgan Freeman")
    genre: str = Field(..., example="Drama")
    plot: str = Field(..., example="Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.")
    runtime: int = Field(..., example=142)

# Définir le modèle de données de sortie
class PredictionResponse(BaseModel):
    """Modèle pour les réponses de prédiction."""
    prediction: float
    confidence: float = None
    model_info: Dict[str, Any]
    features_used: List[str]
    processing_time: float

# Définir l'application FastAPI
app = FastAPI(
    title="Film Rating Predictor API",
    description="API pour prédire la note IMDb d'un film",
    version="1.0.0"
)

# Charger le modèle (dans un environnement réel, vous pourriez le faire au démarrage de l'application)
def load_model():
    model_path = os.path.join(MODELS_DIR, 'demo_imdb_rating_predictor.joblib')
    if os.path.exists(model_path):
        return joblib.load(model_path)
    else:
        # Créer un modèle simple si aucun n'existe
        from sklearn.ensemble import RandomForestRegressor
        model = RandomForestRegressor(n_estimators=50, random_state=42)
        X = np.random.rand(100, 5)
        y = 2 * X[:, 0] + 3 * X[:, 1] - X[:, 2] + np.random.normal(0, 0.1, 100)
        model.fit(X, y)
        return model

# Définir la route pour la page d'accueil
@app.get("/")
async def root():
    return {"message": "Bienvenue sur l'API de prédiction de notes de films"}

# Définir la route pour les prédictions
@app.post("/predict/rating", response_model=PredictionResponse)
async def predict_rating(movie: MovieInput):
    try:
        # Charger le modèle
        model = load_model()
        
        # Dans un cas réel, vous effectueriez ici toutes les transformations nécessaires
        # pour convertir les données d'entrée en format adapté au modèle
        
        # Simuler des caractéristiques extraites
        features = np.array([
            [
                2023 - movie.year,               # age du film
                len(movie.actors.split(',')),    # nombre d'acteurs
                movie.runtime,                   # durée
                len(movie.plot),                 # longueur du résumé
                len(movie.genre.split(','))      # nombre de genres
            ]
        ])
        
        # Faire une prédiction
        import time
        start_time = time.time()
        prediction = model.predict(features)[0]
        end_time = time.time()
        
        # Préparer la réponse
        return {
            "prediction": float(prediction),
            "confidence": 0.85,  # Valeur fictive pour l'exemple
            "model_info": {
                "model_type": "RandomForestRegressor",
                "version": "1.0"
            },
            "features_used": ["movie_age", "actor_count", "runtime", "plot_length", "genre_count"],
            "processing_time": end_time - start_time
        }
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Erreur lors de la prédiction: {str(e)}")

# Afficher un message d'information (dans un environnement réel, vous lanceriez l'API avec uvicorn)
print("Dans un environnement réel, vous lanceriez l'API avec la commande :")
print("uvicorn main:app --reload")
print("\nL'API serait alors accessible à l'adresse : http://localhost:8000")
print("La documentation Swagger serait disponible à : http://localhost:8000/docs")

### 6.2 Simulation de requêtes à l'API

Simulons des requêtes à notre API (sans avoir besoin de la démarrer réellement) :

In [ ]:
# Simuler des requêtes à l'API en appelant directement les fonctions

# Créer quelques exemples de films
sample_movies = [
    MovieInput(
        title="The Shawshank Redemption",
        year=1994,
        director="Frank Darabont",
        actors="Tim Robbins, Morgan Freeman",
        genre="Drama",
        plot="Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.",
        runtime=142
    ),
    MovieInput(
        title="Inception",
        year=2010,
        director="Christopher Nolan",
        actors="Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page",
        genre="Action, Adventure, Sci-Fi",
        plot="A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O.",
        runtime=148
    ),
    MovieInput(
        title="The Godfather",
        year=1972,
        director="Francis Ford Coppola",
        actors="Marlon Brando, Al Pacino, James Caan",
        genre="Crime, Drama",
        plot="The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.",
        runtime=175
    )
]

# Simuler des prédictions pour chaque film
for i, movie in enumerate(sample_movies):
    print(f"\n--- Film {i+1}: {movie.title} ({movie.year}) ---")
    
    # Appeler directement la fonction de prédiction
    response = await predict_rating(movie)
    
    # Afficher les résultats
    print(f"Note prédite: {response['prediction']:.2f}/10")
    print(f"Confiance: {response['confidence']:.2f}")
    print(f"Temps de traitement: {response['processing_time']*1000:.2f} ms")
    print(f"Caractéristiques utilisées: {', '.join(response['features_used'])}")

# Créer un graphique comparant les notes prédites
predictions = []
titles = []

for movie in sample_movies:
    response = await predict_rating(movie)
    predictions.append(response['prediction'])
    titles.append(f"{movie.title} ({movie.year})")

# Visualiser les prédictions
plt.figure(figsize=(12, 6))
bars = plt.bar(titles, predictions)
plt.ylim(0, 10)
plt.xlabel('Film')
plt.ylabel('Note IMDb prédite')
plt.title('Prédictions de notes IMDb pour différents films')

# Ajouter les valeurs au-dessus des barres
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{height:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 7. Optimisation des modèles avec Cross-Validation et Hyperparameter Tuning

Dans cette section, nous allons explorer comment optimiser les modèles de machine learning en utilisant la validation croisée (cross-validation) et l'optimisation des hyperparamètres. Ces techniques sont essentielles pour éviter le surapprentissage et obtenir les meilleures performances possibles.

In [ ]:
# Importer les bibliothèques nécessaires
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Créer un dataset synthétique pour l'exemple
np.random.seed(42)
n_samples = 200
X = np.random.rand(n_samples, 5)
# Créer une variable cible avec du bruit
y = 5 + 2*X[:, 0] + 3*X[:, 1] - 1.5*X[:, 2] + 0.5*X[:, 3] + np.random.normal(0, 0.5, n_samples)

# Convertir en DataFrame pour une meilleure lisibilité
feature_names = ['film_age', 'num_awards', 'runtime_min', 'budget_millions', 'marketing_score']
X_df = pd.DataFrame(X, columns=feature_names)

print("Jeu de données synthétique créé pour la démonstration :")
print(f"Nombre d'échantillons: {n_samples}")
print(f"Caractéristiques: {', '.join(feature_names)}")

# Visualiser les relations entre les caractéristiques et la cible
plt.figure(figsize=(15, 10))
for i, feature in enumerate(feature_names):
    plt.subplot(2, 3, i+1)
    plt.scatter(X_df[feature], y, alpha=0.5)
    plt.title(f'Relation entre {feature} et la note')
    plt.xlabel(feature)
    plt.ylabel('Note')
    plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Validation croisée sur un modèle de base
print("\n1. Validation croisée sur un modèle de base")
base_model = RandomForestRegressor(random_state=42)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Calculer les scores de validation croisée
cv_scores = cross_val_score(base_model, X, y, cv=cv, scoring='r2')
print(f"Scores R² par fold: {cv_scores}")
print(f"Score R² moyen: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# Création d'un pipeline pour prétraitement + modélisation
print("\n2. Création d'un pipeline avec prétraitement")
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(random_state=42))
])

# Scores avec le pipeline
pipeline_cv_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='r2')
print(f"Scores R² du pipeline par fold: {pipeline_cv_scores}")
print(f"Score R² moyen du pipeline: {pipeline_cv_scores.mean():.4f} ± {pipeline_cv_scores.std():.4f}")

# Optimisation des hyperparamètres avec GridSearchCV
print("\n3. Optimisation des hyperparamètres avec GridSearchCV")
print("Cette opération peut prendre un peu de temps...")

# Définir les paramètres à tester
param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': [2, 5, 10]
}

# Créer le GridSearchCV
grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=cv, 
    scoring='r2', 
    return_train_score=True,
    n_jobs=-1  # Utiliser tous les cœurs disponibles
)

# Mesurer le temps d'exécution
start_time = time.time()
grid_search.fit(X, y)
end_time = time.time()

print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")

# Afficher les meilleurs paramètres
print(f"\nMeilleurs paramètres: {grid_search.best_params_}")
print(f"Meilleur score R²: {grid_search.best_score_:.4f}")

# Créer un DataFrame des résultats pour une analyse plus facile
results = pd.DataFrame(grid_search.cv_results_)

# Sélectionner les colonnes pertinentes
cols = ['param_model__n_estimators', 'param_model__max_depth', 
        'param_model__min_samples_split', 'mean_test_score', 
        'std_test_score', 'mean_train_score', 'std_train_score', 'rank_test_score']
results_df = results[cols].sort_values('rank_test_score')

# Afficher les 5 meilleures combinaisons
print("\nTop 5 des combinaisons de paramètres:")
display(results_df.head())

# Visualiser l'impact des paramètres sur la performance
plt.figure(figsize=(15, 10))

# Impact du nombre d'arbres
plt.subplot(2, 2, 1)
sns.boxplot(x='param_model__n_estimators', y='mean_test_score', data=results)
plt.title('Impact du nombre d\'arbres sur le score R²')
plt.xlabel('Nombre d\'arbres')
plt.ylabel('Score R²')

# Impact de la profondeur maximale
plt.subplot(2, 2, 2)
sns.boxplot(x='param_model__max_depth', y='mean_test_score', data=results)
plt.title('Impact de la profondeur maximale sur le score R²')
plt.xlabel('Profondeur maximale')
plt.ylabel('Score R²')

# Impact du nombre minimum d'échantillons pour diviser
plt.subplot(2, 2, 3)
sns.boxplot(x='param_model__min_samples_split', y='mean_test_score', data=results)
plt.title('Impact du min_samples_split sur le score R²')
plt.xlabel('Valeur de min_samples_split')
plt.ylabel('Score R²')

# Comparer train vs test pour détecter le surapprentissage
plt.subplot(2, 2, 4)
plt.scatter(results['mean_train_score'], results['mean_test_score'], alpha=0.7)
plt.plot([0.5, 1], [0.5, 1], 'r--')  # Ligne d'égalité
plt.xlim(0.8, 1)
plt.ylim(0.8, 1)
plt.title('Train vs Test Scores')
plt.xlabel('Score R² moyen sur train')
plt.ylabel('Score R² moyen sur test')
plt.grid(True)

plt.tight_layout()
plt.show()

# Évaluer le meilleur modèle
best_model = grid_search.best_estimator_
print("\nPerformance du meilleur modèle par validation croisée:")
best_cv_scores = cross_val_score(best_model, X, y, cv=cv, scoring='r2')
print(f"Scores R² par fold: {best_cv_scores}")
print(f"Score R² moyen: {best_cv_scores.mean():.4f} ± {best_cv_scores.std():.4f}")

# Comparer différents algorithmes
print("\n4. Comparaison de différents algorithmes")

models = {
    'Ridge': Ridge(random_state=42),
    'RandomForest': RandomForestRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42)
}

model_results = {}
for name, model in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    scores = cross_val_score(pipeline, X, y, cv=cv, scoring='r2')
    model_results[name] = {
        'scores': scores,
        'mean': scores.mean(),
        'std': scores.std()
    }
    print(f"{name}: R² = {scores.mean():.4f} ± {scores.std():.4f}")

# Visualiser la comparaison des modèles
plt.figure(figsize=(10, 6))
model_names = list(model_results.keys())
means = [model_results[name]['mean'] for name in model_names]
stds = [model_results[name]['std'] for name in model_names]

bars = plt.bar(model_names, means, yerr=stds, capsize=10, alpha=0.7)
plt.ylim(0.7, 1.0)
plt.axhline(y=0.9, color='r', linestyle='--', alpha=0.7)
plt.title('Comparaison des performances des modèles (R²)')
plt.ylabel('Score R²')
plt.grid(True, linestyle='--', alpha=0.7, axis='y')

# Ajouter les valeurs sur les barres
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Conclusion sur l'optimisation
print("\nConclusion sur l'optimisation des modèles:")
print("1. La validation croisée permet d'obtenir une estimation plus robuste des performances")
print("2. L'optimisation des hyperparamètres permet d'améliorer significativement les performances")
print("3. Le choix de l'algorithme a un impact important sur les résultats")
print("4. Il est important de surveiller le surapprentissage en comparant les scores train et test")

## 8. Interprétabilité et Explainabilité des Modèles

L'interprétabilité des modèles est essentielle pour comprendre les prédictions et gagner la confiance des utilisateurs. Dans cette section, nous allons explorer des techniques pour expliquer les prédictions des modèles de machine learning, notamment à l'aide de SHAP (SHapley Additive exPlanations) et d'autres méthodes d'interprétation.

In [ ]:
# Importer les bibliothèques nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import joblib
import os

# Note: Normalement, nous utiliserions également SHAP, mais nous allons simuler
# son comportement pour éviter l'installation dans ce notebook

# Créer un dataset simple pour l'exemple
np.random.seed(42)
n_samples = 100
X = np.random.rand(n_samples, 5)
feature_names = ['film_age', 'num_awards', 'runtime_min', 'budget_millions', 'marketing_score']
X_df = pd.DataFrame(X, columns=feature_names)

# Créer une relation connue entre les caractéristiques et la cible
# avec des coefficients d'importance différents
coefficients = [2.5, 1.8, -0.5, 3.2, 0.9]
y = np.dot(X, coefficients) + np.random.normal(0, 0.5, n_samples)

print("Coefficients réels d'importance des caractéristiques:")
for feature, coef in zip(feature_names, coefficients):
    print(f"{feature}: {coef}")

# Entraîner un modèle de forêt aléatoire
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_df, y)

print("\n1. Importance des caractéristiques basée sur l'impureté (méthode intégrée)")
# Extraire et afficher l'importance des caractéristiques
feature_importances = model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values('Importance', ascending=False)

display(importance_df)

# Visualiser l'importance des caractéristiques
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Importance')
plt.title('Importance des caractéristiques (méthode intégrée)')
plt.gca().invert_yaxis()  # Pour avoir la caractéristique la plus importante en haut
plt.tight_layout()
plt.show()

print("\n2. Importance des caractéristiques par permutation")
# Calculer l'importance par permutation
perm_importance = permutation_importance(model, X_df, y, n_repeats=10, random_state=42)
perm_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': perm_importance.importances_mean,
    'Std': perm_importance.importances_std
}).sort_values('Importance', ascending=False)

display(perm_importance_df)

# Visualiser l'importance par permutation
plt.figure(figsize=(10, 6))
plt.barh(perm_importance_df['Feature'], perm_importance_df['Importance'], 
         xerr=perm_importance_df['Std'], capsize=5)
plt.xlabel('Importance (diminution moyenne de la performance)')
plt.title('Importance des caractéristiques par permutation')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\n3. Graphiques de dépendance partielle (simulation)")
# Simuler des graphiques de dépendance partielle
plt.figure(figsize=(15, 10))

for i, feature in enumerate(feature_names):
    plt.subplot(2, 3, i+1)
    
    # Créer un ensemble de valeurs pour la caractéristique
    feature_values = np.linspace(0, 1, 100)
    
    # Créer un jeu de données où toutes les autres caractéristiques sont à leur moyenne
    X_mean = np.tile(X_df.mean().values, (100, 1))
    
    # Remplacer la caractéristique d'intérêt par les valeurs générées
    X_mean[:, i] = feature_values
    
    # Faire des prédictions
    predictions = model.predict(X_mean)
    
    # Tracer la dépendance partielle
    plt.plot(feature_values, predictions)
    plt.title(f'Dépendance partielle: {feature}')
    plt.xlabel(feature)
    plt.ylabel('Prédiction moyenne')
    plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

print("\n4. Simulation des valeurs SHAP")
# Simuler des valeurs SHAP pour un exemple
np.random.seed(42)

# Sélectionner un exemple aléatoire
sample_idx = np.random.randint(0, n_samples)
sample = X_df.iloc[sample_idx]
prediction = model.predict([sample])[0]

print(f"Exemple sélectionné (index {sample_idx}):")
for feature, value in sample.items():
    print(f"{feature}: {value:.4f}")
print(f"Prédiction: {prediction:.4f}")

# Simuler des valeurs SHAP
# Note: Dans un cas réel, nous utiliserions:
# import shap
# explainer = shap.TreeExplainer(model)
# shap_values = explainer.shap_values(sample)

# Simuler des valeurs SHAP basées sur l'importance des caractéristiques
base_value = model.predict(np.mean(X_df, axis=0).reshape(1, -1))[0]
contributions = []

for i, (feature, value) in enumerate(sample.items()):
    # Simuler une contribution basée sur l'écart par rapport à la moyenne et l'importance
    contribution = (value - X_df[feature].mean()) * feature_importances[i] * 5
    contributions.append(contribution)

# Normaliser pour que la somme corresponde à l'écart de prédiction
contributions = np.array(contributions)
scale_factor = (prediction - base_value) / contributions.sum()
shap_values = contributions * scale_factor

# Créer un DataFrame des valeurs SHAP
shap_df = pd.DataFrame({
    'Feature': feature_names,
    'SHAP Value': shap_values,
    'Feature Value': sample.values
}).sort_values('SHAP Value', ascending=False)

print("\nValeurs SHAP simulées:")
display(shap_df)

# Visualiser les valeurs SHAP
plt.figure(figsize=(10, 6))
colors = ['red' if x > 0 else 'blue' for x in shap_df['SHAP Value']]
plt.barh(shap_df['Feature'], shap_df['SHAP Value'], color=colors)
plt.axvline(x=0, color='gray', linestyle='--')
plt.xlabel('Impact sur la prédiction (valeur SHAP)')
plt.title(f'Explication de la prédiction (valeur de base: {base_value:.4f}, prédiction: {prediction:.4f})')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\n5. Force plot simulé")
# Créer un graphique de force simulé pour montrer l'impact cumulatif
plt.figure(figsize=(12, 4))

# Trier les valeurs SHAP par ordre absolu décroissant
shap_order = np.argsort(np.abs(shap_values))[::-1]
ordered_features = [feature_names[i] for i in shap_order]
ordered_shap = shap_values[shap_order]

# Calculer les valeurs cumulatives
cumulative = np.cumsum(ordered_shap) + base_value
all_values = np.concatenate(([base_value], cumulative))

# Tracer le graphique en cascade
plt.plot([0, len(ordered_features) + 1], [base_value, prediction], 'k--', alpha=0.5)
plt.scatter(range(1, len(ordered_features) + 1), cumulative, s=50)

# Ajouter des annotations
for i, (feature, shap_val) in enumerate(zip(ordered_features, ordered_shap)):
    x = i + 1
    y = all_values[i]
    direction = 'up' if shap_val > 0 else 'down'
    color = 'green' if shap_val > 0 else 'red'
    plt.annotate(
        f"{feature} ({shap_val:.2f})",
        xy=(x, all_values[i+1]),
        xytext=(x, all_values[i+1] + (0.1 if direction == 'up' else -0.1)),
        arrowprops=dict(arrowstyle='->', color=color),
        ha='center'
    )
    plt.plot([x, x], [y, all_values[i+1]], color=color)

plt.xticks([0] + list(range(1, len(ordered_features) + 1)), ['base'] + ordered_features, rotation=45, ha='right')
plt.ylabel('Prédiction')
plt.title('Force Plot: Impact cumulatif des caractéristiques sur la prédiction')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

print("\nCette section illustre différentes méthodes pour interpréter les modèles :")
print("1. Importance des caractéristiques basée sur l'impureté (intégrée aux arbres de décision)")
print("2. Importance par permutation (plus robuste car basée sur la dégradation des performances)")
print("3. Graphiques de dépendance partielle (montrent comment une caractéristique influence la prédiction)")
print("4. Valeurs SHAP (attribuent une contribution à chaque caractéristique pour une prédiction)")
print("5. Force plots (visualisent l'impact cumulatif des caractéristiques)")
print("\nDans un environnement réel, il serait recommandé d'utiliser la bibliothèque SHAP pour des explications plus précises.")

## 9. Surveillance et Évaluation Continue

Une fois un modèle déployé, il est crucial de le surveiller et de l'évaluer en continu pour s'assurer qu'il fonctionne correctement et que ses performances ne se dégradent pas au fil du temps. Dans cette section, nous allons explorer quelques techniques pour surveiller et évaluer les modèles en production.

In [ ]:
# Importer les bibliothèques nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
from datetime import datetime, timedelta

print("Simulation d'un système de surveillance et d'évaluation continue des modèles")

# Créer un dataset synthétique pour l'exemple
np.random.seed(42)
n_samples = 500
X = np.random.rand(n_samples, 5)
feature_names = ['film_age', 'num_awards', 'runtime_min', 'budget_millions', 'marketing_score']
X_df = pd.DataFrame(X, columns=feature_names)

# Créer une relation connue entre les caractéristiques et la cible
y = 5 + 2*X[:, 0] + 3*X[:, 1] - 1.5*X[:, 2] + 0.5*X[:, 3] + np.random.normal(0, 0.5, n_samples)

# Division en train/test chronologique (simulé)
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.4, shuffle=False)

print(f"Données d'entraînement: {X_train.shape[0]} exemples")
print(f"Données de test: {X_test.shape[0]} exemples")

# Entraîner un modèle initial
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("\n1. Surveillance des performances sur le temps")
print("Simulation d'une évaluation sur 10 semaines...")

# Diviser les données de test en 10 semaines
test_splits = np.array_split(list(range(len(X_test))), 10)
weeks = range(1, 11)
weekly_metrics = []

# Simuler un drift graduel dans les données
drift_factor = np.linspace(0, 0.5, 10)  # Augmentation graduelle du drift

for week, split, drift in zip(weeks, test_splits, drift_factor):
    X_week = X_test.iloc[split]
    y_week = y_test.iloc[split]
    
    # Appliquer un drift simulé aux caractéristiques
    X_week_drift = X_week.copy()
    X_week_drift['marketing_score'] = X_week_drift['marketing_score'] + drift
    
    # Faire des prédictions
    y_pred = model.predict(X_week_drift)
    
    # Calculer les métriques
    rmse = np.sqrt(mean_squared_error(y_week, y_pred))
    mae = mean_absolute_error(y_week, y_pred)
    r2 = r2_score(y_week, y_pred)
    
    weekly_metrics.append({
        'Week': week,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'Drift': drift
    })

# Convertir en DataFrame
metrics_df = pd.DataFrame(weekly_metrics)
display(metrics_df)

# Visualiser l'évolution des métriques
plt.figure(figsize=(15, 10))

# RMSE au fil du temps
plt.subplot(2, 2, 1)
plt.plot(metrics_df['Week'], metrics_df['RMSE'], 'o-', linewidth=2)
plt.axhline(y=np.mean(metrics_df['RMSE']), color='r', linestyle='--', alpha=0.7)
plt.fill_between(
    metrics_df['Week'], 
    np.mean(metrics_df['RMSE']) - np.std(metrics_df['RMSE']), 
    np.mean(metrics_df['RMSE']) + np.std(metrics_df['RMSE']), 
    alpha=0.2, color='r'
)
plt.title('RMSE au fil du temps')
plt.xlabel('Semaine')
plt.ylabel('RMSE')
plt.grid(True)

# MAE au fil du temps
plt.subplot(2, 2, 2)
plt.plot(metrics_df['Week'], metrics_df['MAE'], 'o-', linewidth=2, color='green')
plt.axhline(y=np.mean(metrics_df['MAE']), color='r', linestyle='--', alpha=0.7)
plt.fill_between(
    metrics_df['Week'], 
    np.mean(metrics_df['MAE']) - np.std(metrics_df['MAE']), 
    np.mean(metrics_df['MAE']) + np.std(metrics_df['MAE']), 
    alpha=0.2, color='r'
)
plt.title('MAE au fil du temps')
plt.xlabel('Semaine')
plt.ylabel('MAE')
plt.grid(True)

# R² au fil du temps
plt.subplot(2, 2, 3)
plt.plot(metrics_df['Week'], metrics_df['R2'], 'o-', linewidth=2, color='purple')
plt.axhline(y=np.mean(metrics_df['R2']), color='r', linestyle='--', alpha=0.7)
plt.fill_between(
    metrics_df['Week'], 
    np.mean(metrics_df['R2']) - np.std(metrics_df['R2']), 
    np.mean(metrics_df['R2']) + np.std(metrics_df['R2']), 
    alpha=0.2, color='r'
)
plt.title('R² au fil du temps')
plt.xlabel('Semaine')
plt.ylabel('R²')
plt.grid(True)

# Relation entre le drift et l'erreur
plt.subplot(2, 2, 4)
plt.scatter(metrics_df['Drift'], metrics_df['RMSE'], alpha=0.7, s=100)
plt.title('Impact du Drift sur l\'erreur')
plt.xlabel('Facteur de Drift')
plt.ylabel('RMSE')
plt.grid(True)

plt.tight_layout()
plt.show()

print("\n2. Simulation d'une détection de drift de données")

# Créer des données avec drift pour la simulation
np.random.seed(43)
n_drift_samples = 200
X_new = np.random.rand(n_drift_samples, 5) * 1.2 - 0.1  # Légèrement décalé
X_new_df = pd.DataFrame(X_new, columns=feature_names)

# Analyser la distribution des caractéristiques
plt.figure(figsize=(15, 10))
for i, feature in enumerate(feature_names):
    plt.subplot(2, 3, i+1)
    
    # Distributions originales et nouvelles
    sns.kdeplot(X_df[feature], label='Original', alpha=0.7)
    sns.kdeplot(X_new_df[feature], label='Nouveaux', alpha=0.7)
    
    plt.title(f'Distribution de {feature}')
    plt.xlabel(feature)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Calcul de la divergence entre distributions (approximation simple)
def distribution_divergence(dist1, dist2, bins=20):
    """Calcule une approximation simple de la divergence entre deux distributions."""
    hist1, edges = np.histogram(dist1, bins=bins, density=True)
    hist2, _ = np.histogram(dist2, bins=edges, density=True)
    
    # Éviter la division par zéro
    hist1 = np.maximum(hist1, 1e-10)
    hist2 = np.maximum(hist2, 1e-10)
    
    # Approximation de la divergence KL
    kl_div = np.sum(hist1 * np.log(hist1 / hist2))
    return kl_div

# Calculer la divergence pour chaque caractéristique
divergences = {}
for feature in feature_names:
    div = distribution_divergence(X_df[feature], X_new_df[feature])
    divergences[feature] = div

# Afficher les divergences
divergences_df = pd.DataFrame({
    'Feature': list(divergences.keys()),
    'Divergence': list(divergences.values())
}).sort_values('Divergence', ascending=False)

print("Divergence entre les distributions originales et nouvelles:")
display(divergences_df)

# Visualiser les divergences
plt.figure(figsize=(10, 6))
plt.bar(divergences_df['Feature'], divergences_df['Divergence'])
plt.title('Divergence entre les distributions originales et nouvelles')
plt.ylabel('Divergence (approx. KL)')
plt.xticks(rotation=45)
plt.axhline(y=0.5, color='r', linestyle='--', label='Seuil d\'alerte')
plt.legend()
plt.tight_layout()
plt.show()

print("\n3. Simulation d'un système de surveillance des performances")

# Simuler un tableau de bord de surveillance
print("Tableau de bord de surveillance des modèles")
print("------------------------------------------")

# Métriques générales
avg_rmse = metrics_df['RMSE'].mean()
last_rmse = metrics_df['RMSE'].iloc[-1]
rmse_change = (last_rmse - avg_rmse) / avg_rmse * 100

avg_r2 = metrics_df['R2'].mean()
last_r2 = metrics_df['R2'].iloc[-1]
r2_change = (last_r2 - avg_r2) / avg_r2 * 100

# Afficher les métriques principales
print(f"RMSE moyenne: {avg_rmse:.4f}")
print(f"RMSE actuelle: {last_rmse:.4f} ({'+' if rmse_change > 0 else ''}{rmse_change:.2f}%)")
print(f"R² moyen: {avg_r2:.4f}")
print(f"R² actuel: {last_r2:.4f} ({'+' if r2_change > 0 else ''}{r2_change:.2f}%)")

# Statut du modèle
if rmse_change > 10 or r2_change < -10:
    status = "ALERTE"
elif rmse_change > 5 or r2_change < -5:
    status = "AVERTISSEMENT"
else:
    status = "NORMAL"

print(f"Statut du modèle: {status}")

# Recommandations
print("\nRecommandations:")
if status == "ALERTE":
    print("- Réentraîner le modèle immédiatement avec les données récentes")
    print("- Vérifier la qualité des données entrantes")
    print("- Revoir les caractéristiques impactées par le drift")
elif status == "AVERTISSEMENT":
    print("- Prévoir un réentraînement dans la semaine")
    print("- Surveiller de près les caractéristiques avec drift élevé")
    print("- Analyser les prédictions erronées")
else:
    print("- Continuer la surveillance régulière")
    print("- Planifier le réentraînement mensuel standard")

# Simuler un historique des réentraînements
print("\nHistorique des réentraînements:")
today = datetime.now()

retraining_history = [
    {"date": (today - timedelta(days=60)).strftime("%Y-%m-%d"), "version": "1.0", "reason": "Initial deployment"},
    {"date": (today - timedelta(days=45)).strftime("%Y-%m-%d"), "version": "1.1", "reason": "Scheduled update"},
    {"date": (today - timedelta(days=30)).strftime("%Y-%m-%d"), "version": "1.2", "reason": "Feature improvement"},
    {"date": (today - timedelta(days=15)).strftime("%Y-%m-%d"), "version": "1.3", "reason": "Data drift correction"}
]

for entry in retraining_history:
    print(f"- {entry['date']}: v{entry['version']} - {entry['reason']}")

print("\nProchain réentraînement prévu: " + (today + timedelta(days=15)).strftime("%Y-%m-%d"))

print("\nCette section a démontré :")
print("1. Comment surveiller les performances d'un modèle au fil du temps")
print("2. Comment détecter et mesurer le drift dans les données")
print("3. Comment implémenter un système d'alerte pour la dégradation des performances")
print("4. L'importance de la planification des réentraînements réguliers")
print("\nEn production, ces mécanismes seraient automatisés et intégrés à un pipeline de MLOps.")

## 10. Conclusion et Bonnes Pratiques

Dans ce notebook, nous avons couvert l'ensemble du pipeline de data engineering et de machine learning, de la collecte des données au déploiement d'une API. Voici un récapitulatif des étapes et des bonnes pratiques pour chacune d'entre elles :

### 1. Collecte de données
- Utiliser différentes sources (APIs, webscraping, fichiers locaux)
- Implémenter une gestion des erreurs robuste
- Documenter la provenance des données

### 2. Nettoyage des données
- Traiter les valeurs manquantes de manière adaptée au contexte
- Normaliser les formats de données
- Détecter et gérer les outliers
- Effectuer des vérifications de cohérence

### 3. Feature Engineering
- Créer des caractéristiques pertinentes pour le problème
- Encoder les variables catégorielles
- Normaliser/standardiser les variables numériques
- Extraire des caractéristiques textuelles (sentiments, TF-IDF, etc.)

### 4. Entraînement de modèles
- Tester différents algorithmes
- Optimiser les hyperparamètres
- Utiliser la validation croisée
- Évaluer avec des métriques adaptées

### 5. Stockage et chargement de modèles
- Utiliser des formats standardisés (joblib, pickle)
- Stocker les métadonnées (performances, paramètres, etc.)
- Versionner les modèles

### 6. Déploiement d'API
- Utiliser des frameworks modernes (FastAPI)
- Documenter l'API (Swagger, OpenAPI)
- Gérer les erreurs
- Optimiser les performances

### 7. Optimisation des modèles
- Utiliser la validation croisée
- Appliquer l'optimisation des hyperparamètres
- Surveiller le surapprentissage

### 8. Interprétabilité des modèles
- Analyser l'importance des caractéristiques
- Utiliser des méthodes d'explicabilité (SHAP, LIME)
- Visualiser les dépendances partielles

### 9. Surveillance et évaluation continue
- Monitorer les performances au fil du temps
- Détecter les drifts de données
- Planifier des réentraînements réguliers

Ce pipeline complet illustre les étapes essentielles pour développer et déployer des modèles de machine learning en production. Il peut être adapté à différents cas d'usage et étendu selon les besoins spécifiques.

La mise en place d'un tel pipeline nécessite une bonne compréhension des concepts de data engineering et de machine learning, ainsi qu'une maîtrise des outils et technologies associés.

In [ ]:
# Afficher quelques ressources supplémentaires et prochaines étapes

print("Ressources pour approfondir :")
print("-----------------------------")

resources = [
    {"topic": "Data Engineering", "resources": [
        "Apache Airflow - https://airflow.apache.org/",
        "dbt (data build tool) - https://www.getdbt.com/",
        "Great Expectations - https://greatexpectations.io/"
    ]},
    {"topic": "Machine Learning", "resources": [
        "scikit-learn - https://scikit-learn.org/",
        "XGBoost - https://xgboost.readthedocs.io/",
        "TensorFlow - https://www.tensorflow.org/"
    ]},
    {"topic": "MLOps", "resources": [
        "MLflow - https://mlflow.org/",
        "DVC - https://dvc.org/",
        "Kubeflow - https://www.kubeflow.org/"
    ]},
    {"topic": "Explainabilité", "resources": [
        "SHAP - https://github.com/slundberg/shap",
        "LIME - https://github.com/marcotcr/lime",
        "ELI5 - https://eli5.readthedocs.io/"
    ]},
    {"topic": "API & Déploiement", "resources": [
        "FastAPI - https://fastapi.tiangolo.com/",
        "Docker - https://www.docker.com/",
        "Kubernetes - https://kubernetes.io/"
    ]}
]

for category in resources:
    print(f"\n{category['topic']}:")
    for resource in category['resources']:
        print(f"- {resource}")

print("\nProchaines étapes possibles pour ce projet :")
print("-------------------------------------------")
next_steps = [
    "Implémenter des tests unitaires pour chaque composant du pipeline",
    "Ajouter un système de versionnage des données avec DVC",
    "Automatiser le pipeline complet avec Apache Airflow",
    "Conteneuriser l'application avec Docker",
    "Implémenter un système de feedback pour améliorer le modèle",
    "Ajouter des méthodes d'explainabilité avancées (SHAP, LIME)",
    "Mettre en place une surveillance des performances en temps réel",
    "Développer une interface utilisateur pour visualiser les prédictions",
    "Implémenter un système de A/B testing pour les nouveaux modèles",
    "Intégrer une base de données pour stocker les prédictions et feedback"
]

for i, step in enumerate(next_steps):
    print(f"{i+1}. {step}")

print("\nMerci d'avoir suivi ce notebook sur le pipeline complet de data engineering et machine learning !")
print("N'hésitez pas à adapter et étendre ce code pour vos propres projets.")